# Linear Regression Functions

In [5]:
from setup import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
import statsmodels.api as sm
config = load_config()

In [2]:
#@title Figure settings
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

## **Plotting**

In [1]:
def plot_connectivity_vs_behavior(X_summary, y, model=None):
    """
    Plot connectivity summary features against behavioral measure and model line.
    
    Parameters:
    X_summary (numpy array): Summary feature matrix.
    y (numpy array): Target vector.
    model: Trained linear regression model (optional).
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(X_summary, y, color='blue', label='Observed data')
    
    if model is not None:
        # Predict using the model
        y_pred = model.predict(X_summary)
        plt.plot(X_summary, y_pred, color='red', linewidth=2, label='Model prediction')
        slope = model.coef_[0]
        intercept = model.intercept_
        
        # Display model parameters on the plot
        plt.text(0.05, 0.95, f'Slope: {slope:.4f}\nIntercept: {intercept:.4f}', 
                 transform=plt.gca().transAxes, fontsize=12,
                 verticalalignment='top', bbox=dict(boxstyle='round,pad=0.3', edgecolor='black', facecolor='white'))
    
    plt.xlabel('Connectivity Summary Features')
    plt.ylabel('Behavioral Measure')
    plt.title('Connectivity Summary Features vs. Behavioral Measure')
    plt.legend()
    plt.grid(True)
    plt.show()

## **Training and Fitting**

In [2]:
def train_linear_regression(X_train, y_train):
    """
    Trains a linear regression model.

    Parameters:
    X_train (numpy array): Training feature matrix.
    y_train (numpy array): Training target vector.

    Returns:
    model: Trained linear regression model.
    """
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model

def predict(model, X_test):
    """
    Makes predictions using the trained linear regression model.

    Parameters:
    model: Trained linear regression model.
    X_test (numpy array): Testing feature matrix.

    Returns:
    y_pred (numpy array): Predicted target vector.
    """
    y_pred = model.predict(X_test)
    return y_pred

## **Evaluation Methods**

In [1]:
def split_data(X, y, test_size=0.2, random_state=42):
    """
    Splits the data into training and testing sets.

    Parameters:
    X (numpy array): Feature matrix.
    y (numpy array): Target vector.
    test_size (float): Proportion of the dataset to include in the test split.
    random_state (int): Seed used by the random number generator.

    Returns:
    X_train, X_test, y_train, y_test: Split data.
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

def evaluate_model_mse(y_test, y_pred):
    """
    Evaluates the model performance.

    Parameters:
    y_test (numpy array): True target vector.
    y_pred (numpy array): Predicted target vector.

    Returns:
    mse (float): Mean Squared Error.
    r2 (float): R-squared value.
    """
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, r2

def compute_p_values(X, y):
    """
    Compute p-values for the coefficients of a linear regression model.
    
    Parameters:
    X (numpy array): Feature matrix.
    y (numpy array): Target vector.
    
    Returns:
    p_values (numpy array): p-values for the coefficients.
    """
    # Add a constant (intercept) to the model
    X = sm.add_constant(X)
    
    # Fit the model
    model = sm.OLS(y, X).fit()
    
    # Get p-values
    p_values = model.pvalues
    return p_values

def kfold_cross_validation(X, y, k=10, plotting=True):
    """
    Performs 10-fold cross-validation for linear regression.

    Parameters:
    X (numpy array): Feature matrix.
    y (numpy array): Target vector.
    k (int): Number of folds. Default is 10.

    Returns:
    float: Mean R^2 score across folds.
    """
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    r_scores = []
    mse_scores = []
    p_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model = train_linear_regression(X_train, y_train)
        y_pred = predict(model, X_test)
        if plotting:
            plot_connectivity_vs_behavior(X_summary.reshape(-1, 1), y, model)
        
        mse, r2 = evaluate_model_mse(y_test, y_pred)
        p_value = compute_p_values(X_train, y_train)
        r = np.sqrt(r2) if r2 > 0 else 0
        r_scores.append(r)
        mse_scores.append(mse)
        p_scores.append(p_value[1:])
    
    mean_r = np.mean(r_scores)
    mean_mse = np.mean(mse_scores)
    mean_p_values = np.mean(np.vstack(p_scores), axis=0)
    return mean_r, mean_mse, mean_p_values